In [35]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import PolynomialFeatures
import category_encoders as ce
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, BatchNormalization
from tensorflow.keras.models import Model
from sklearn.metrics.pairwise import cosine_similarity
from keras.optimizers import RMSprop
from keras.callbacks import LearningRateScheduler

In [2]:
signdata = pd.read_csv('/Users/emilkoch/Library/Mobile Documents/com~apple~CloudDocs/Data Files/signdata.csv', encoding='latin-1')

In [4]:
# Separate target variable from features
X = signdata.drop(columns=['SignBankEnglishTranslations'])  # Features
y = signdata['SignBankEnglishTranslations']

# Separate numerical and categorical columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

print("Numerical Columns:", numerical_cols)
print("Categorical Columns:", categorical_cols)

Numerical Columns: ['List', 'Item', 'EnglishWF(lg10)', 'SignFrequency(M)', 'SignFrequency(SD)', 'SignFrequency(Z)', 'SignFrequency(N)', 'Unknown', 'SignFrequency(M-Native)', 'SignFrequency(SD-Native)', 'SignFreq(Z-native)', 'SignFrequency(N-Native)', 'Unknown(Native)', 'SignFrequency(M-Nonnative)', 'SignFrequency(SD-Nonnative)', 'SignFrequency(N-Nonnative)', 'SignFreq(Z-Nonnative)', 'Unknown(Nonnative)', 'DominantTranslationAgreement', 'DominantTranslationAgreement(Native)', 'DominantTranslationAgreement(Nonnative)', 'Iconicity(M)', 'Iconicity(SD)', 'Iconicity(Z)', 'Iconicity(N)', 'D.Iconicity(M)', 'D.Iconicity(SD)', 'D.Iconicity(N)', 'D.Iconicity(Z)', 'D.Iconicity(M-native)', 'D.Iconicity(SD-native)', 'D.Iconicity(Z-native)', 'D.Iconicity(N-native)', 'GuessConsistency', 'GuessAccuracy', 'Transparency(M)', 'Transparency SD', 'Transparency Z', 'Initialized.2.0', 'FingerspelledLoanSign.2.0', 'Compound.2.0', 'NumberOfMorphemes.2.0', 'SignOnset(ms)', 'SignOffset(ms)', 'SignDuration(ms)', '

In [5]:
# Preprocessing for numerical features
numerical_imputer = SimpleImputer(strategy='median')
scaler = StandardScaler()

# Copy numerical columns
X_numerical = X[numerical_cols].copy()
print(len(X_numerical))
print(len(numerical_cols))
print(X_numerical.head())
print(X_numerical.isnull().sum())

2723
129
   List  Item  EnglishWF(lg10)  SignFrequency(M)  SignFrequency(SD)  \
0     1     2            3.521             5.143              2.081   
1     1     3            4.645             6.032              1.516   
2     1     4            2.600             4.429              1.720   
3     1     5            2.928             2.621              1.720   
4     1     8            3.041             1.579              0.838   

   SignFrequency(Z)  SignFrequency(N)  Unknown  SignFrequency(M-Native)  \
0             0.621                21    0.000                    5.167   
1             1.068                31    0.000                    6.111   
2             0.232                21    0.000                    4.167   
3            -0.753                29    0.065                    2.000   
4            -1.198                19    0.095                    1.455   

   SignFrequency(SD-Native)  ...  ThumbContact.2.0Frequency  \
0                     2.167  ...                  

In [6]:
# Impute missing values and scaling
imputer = SimpleImputer(strategy='median')
X_numerical_imputed = imputer.fit_transform(X_numerical) 
scaler = StandardScaler()
X_numerical_scaled  = scaler.fit_transform(X_numerical_imputed)  

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/impute/_base.py:555: UserWarning: Skipping features without any observed values: ['UlnarRotationM4.2.0' 'FlexionChangeM5.2.0' 'SpreadChangeM5.2.0'
 'SignTypeM5.2.0' 'MovementM5.2.0' 'RepeatedMovementM5.2.0'
 'MajorLocationM5.2.0' 'MinorLocationM5.2.0' 'SecondMinorLocationM5.2.0'
 'ContactM5.2.0' 'NonDominantHandshapeM5.2.0' 'UlnarRotationM5.2.0'
 'FlexionChangeM6.2.0' 'SpreadChangeM6.2.0' 'SignTypeM6.2.0'
 'MovementM6.2.0' 'RepeatedMovementM6.2.0' 'MajorLocationM6.2.0'
 'MinorLocationM6.2.0' 'SecondMinorLocationM6.2.0' 'ContactM6.2.0'
 'NonDominantHandshapeM6.2.0' 'UlnarRotationM6.2.0']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


In [7]:
# Preprocessing for categorical features
categorical_imputer = SimpleImputer(strategy='most_frequent')
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

# Copy categorical columns
X_categorical = X[categorical_cols].copy()

# Impute missing values in categorical columns
X_categorical = pd.DataFrame(categorical_imputer.fit_transform(X_categorical), columns=X_categorical.columns)

# Encode categorical features
encoded_cols = pd.DataFrame(encoder.fit_transform(X_categorical))
encoded_cols.columns = encoder.get_feature_names_out(categorical_cols)
categorical_cols_encoded = encoded_cols.columns.tolist()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [8]:
# Concatenate numerical and encoded categorical columns
X_processed = pd.concat([pd.DataFrame(X_numerical_scaled), encoded_cols], axis=1)

In [9]:
# Check for NaN values in the target variable
nan_count = signdata['SignBankEnglishTranslations'].isnull().sum()
print("Number of NaN values in 'SignBankEnglishTranslations' column:", nan_count)

# Initialize SimpleImputer
imputer = SimpleImputer(strategy='most_frequent')

# Apply SimpleImputer to fill missing values in the target variable
filled_values = imputer.fit_transform(signdata[['SignBankEnglishTranslations']])
y_imputed = filled_values.flatten()  # Flatten the 2D array to 1D before assigning back to the Series

# Check for NaN values in the target variable after imputation
nan_count_after_impute = pd.Series(y_imputed).isnull().sum()
print("Number of NaN values in 'SignBankEnglishTranslations' column after imputation:", nan_count_after_impute)
print(y_imputed.dtype)

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Apply label encoding to the target variable 'SignBankEnglishTranslations'
y_encoded = label_encoder.fit_transform(y_imputed)
print(y_encoded.dtype)

Number of NaN values in 'SignBankEnglishTranslations' column: 739
Number of NaN values in 'SignBankEnglishTranslations' column after imputation: 0
object
int64


In [10]:
# Initialize Random Forest Classifier
rf_classifier = RandomForestClassifier()

# Convert feature names to strings
X_processed.columns = [str(col) for col in X_processed.columns]

# Fit the classifier to your data
rf_classifier.fit(X_processed, y_encoded)

# Get feature importances
feature_importances = rf_classifier.feature_importances_

# Filter features based on the threshold
selected_features = X_processed.columns[feature_importances > 0.01]
print(selected_features.isna().sum())

0


In [11]:
# Calculate pairwise correlation between 'SignBankEnglishTranslations' and other numerical columns
correlation_with_target = X_processed.corrwith(pd.Series(y_encoded))

# Sort correlations in descending order
correlation_with_target_sorted = correlation_with_target.abs().sort_values(ascending=False)

# Print the correlation values
print("Pairwise Correlation with 'SignBankEnglishTranslations':")
print(correlation_with_target_sorted)

# Fill NaN values with 0
X_processed_filled = X_processed.fillna(0)

# Filter correlations greater than 0.01
significant_correlations = correlation_with_target_sorted[correlation_with_target_sorted.abs() > 0.01]

# Create a new DataFrame to store significant correlations
significant_correlations_df = pd.DataFrame({'Feature': significant_correlations.index, 'Correlation': significant_correlations.values})

# Print the new DataFrame
print("Significant Correlations (> 0.01):")
print(significant_correlations_df)
print(significant_correlations_df.isna().sum())

Pairwise Correlation with 'SignBankEnglishTranslations':
SignBankLemmaID_DOG                   0.201446
SignBankAnnotationID_SHOUT            0.196797
SignBankSemanticField_Animal          0.158351
CDISemanticCategory_Animals           0.119098
15                                    0.105201
                                        ...   
MovementM4.2.0_Straight                    NaN
SecondMinorLocationM4.2.0_HandAway         NaN
HandshapeM6.2.0_r                          NaN
SelectedFingersM6.2.0_im                   NaN
FlexionM6.2.0_Crossed                      NaN
Length: 14060, dtype: float64
Significant Correlations (> 0.01):
                           Feature  Correlation
0              SignBankLemmaID_DOG     0.201446
1       SignBankAnnotationID_SHOUT     0.196797
2     SignBankSemanticField_Animal     0.158351
3      CDISemanticCategory_Animals     0.119098
4                               15     0.105201
...                            ...          ...
8188    SignBankAnnotatio

In [12]:
# Initialize Random Forest Classifier
rf_classifier = RandomForestClassifier()

# Convert feature names to strings
X_processed.columns = [str(col) for col in X_processed.columns]

# Fit the classifier to your data
rf_classifier.fit(X_processed, y_encoded)

# Get feature importances
feature_importances = rf_classifier.feature_importances_

# Filter features based on the threshold
selected_features_rf = X_processed.columns[feature_importances > 0.01]

# Convert significant correlations to a set for efficient comparison
significant_correlations_set = set(significant_correlations_df['Feature'])

# Convert selected features by random forest to a set for efficient comparison
selected_features_rf_set = set(selected_features_rf)

# Find common features between significant correlations and random forest selected features
common_features = significant_correlations_set.intersection(selected_features_rf_set)

# Print common features
print("Common Features Selected by Random Forest and Significant Correlations (> 0.01):")
print(common_features)

Common Features Selected by Random Forest and Significant Correlations (> 0.01):
{'SignBankAnnotationID_SHOUT', '103', 'SignBankLemmaID_DOG'}


In [15]:
# Define X_train_corr and X_test_corr using the features from significant_correlations_df
X_train_corr = X_train[significant_correlations_df['Feature']]
X_test_corr = X_test[significant_correlations_df['Feature']]

# Define X_train_rf and X_test_rf using the selected features
X_train_rf = X_train[selected_features]
X_test_rf = X_test[selected_features]

# Train Random Forest using significant correlations
rf_corr_classifier = RandomForestClassifier(random_state=42)
rf_corr_classifier.fit(X_train_corr, y_train)

# Train Random Forest using randomly mixed features
rf_rf_classifier = RandomForestClassifier(random_state=42)
rf_rf_classifier.fit(X_train_rf, y_train)

# Predictions
y_pred_corr = rf_corr_classifier.predict(X_test_corr)
y_pred_rf = rf_rf_classifier.predict(X_test_rf)

# Evaluate performance
accuracy_corr = accuracy_score(y_test, y_pred_corr)
accuracy_rf = accuracy_score(y_test, y_pred_rf)

print("Accuracy using features with significant correlations:", accuracy_corr)
print("Accuracy using randomly mixed features:", accuracy_rf)

Accuracy using features with significant correlations: 0.28440366972477066
Accuracy using randomly mixed features: 0.3467889908256881


In [14]:
# Assuming you have your data stored in X_processed and y_encoded
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_processed, y_encoded, test_size=0.2, random_state=42)

# Define the autoencoder architecture
input_dim = X_train.shape[1]
encoding_dim = 64  # Adjust as needed

input_layer = Input(shape=(input_dim,))
encoder_layer = Dense(encoding_dim, activation='relu')(input_layer)
decoder_layer = Dense(input_dim, activation='sigmoid')(encoder_layer)

autoencoder = Model(input_layer, decoder_layer)

# Compile the autoencoder model
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Train the autoencoder
autoencoder.fit(X_train, X_train, epochs=50, batch_size=32, shuffle=True, validation_data=(X_test, X_test))

# Extract features using the encoder part of the autoencoder
encoder = Model(input_layer, encoder_layer)
X_encoded_train = encoder.predict(X_train)
X_encoded_test = encoder.predict(X_test)

# Reconstruct data using the trained autoencoder
reconstructed_data = autoencoder.predict(X_test)

# Combine original test data with reconstructed data
X_test_combined = np.concatenate((X_test, reconstructed_data), axis=1)

# Compute cosine similarity between original and reconstructed data samples
cosine_similarities = cosine_similarity(X_test_combined)

# Calculate the mean cosine similarity across all samples
mean_cosine_similarity = np.mean(cosine_similarities)
print("Mean Cosine Similarity:", mean_cosine_similarity)

Epoch 1/50
69/69 [==============================] - 4s 33ms/step - loss: 0.0751 - val_loss: 0.0079
Epoch 2/50
69/69 [==============================] - 1s 19ms/step - loss: 0.0082 - val_loss: 0.0079
Epoch 3/50
69/69 [==============================] - 2s 23ms/step - loss: 0.0081 - val_loss: 0.0079
Epoch 4/50
69/69 [==============================] - 1s 19ms/step - loss: 0.0081 - val_loss: 0.0078
Epoch 5/50
69/69 [==============================] - 1s 21ms/step - loss: 0.0080 - val_loss: 0.0078
Epoch 6/50
69/69 [==============================] - 1s 19ms/step - loss: 0.0079 - val_loss: 0.0076
Epoch 7/50
69/69 [==============================] - 2s 23ms/step - loss: 0.0078 - val_loss: 0.0074
Epoch 8/50
69/69 [==============================] - 2s 28ms/step - loss: 0.0076 - val_loss: 0.0072
Epoch 9/50
69/69 [==============================] - 2s 22ms/step - loss: 0.0074 - val_loss: 0.0071
Epoch 10/50
69/69 [==============================] - 3s 41ms/step - loss: 0.0073 - val_loss: 0.0069
Epoch 11/

In [33]:
# Define a function to create the autoencoder model
def create_autoencoder(optimizer='adam'):
    # Initialize optimizer based on the provided string
    if optimizer == 'adam':
        optimizer = 'adam'
    elif optimizer == 'rmsprop':
        optimizer = RMSprop()
    elif optimizer == 'adadelta':
        optimizer = Adadelta()
    elif optimizer == 'adagrad':
        optimizer = Adagrad()

    input_layer = Input(shape=(input_dim,))
    encoder_layer = Dense(encoding_dim, activation='relu')(input_layer)
    decoder_layer = Dense(input_dim, activation='sigmoid')(encoder_layer)
    autoencoder = Model(input_layer, decoder_layer)
    autoencoder.compile(optimizer=optimizer, loss='mean_squared_error')
    return autoencoder

# Create the autoencoder model
autoencoder = KerasRegressor(build_fn=create_autoencoder, epochs=50, batch_size=32, verbose=0)

# Define the hyperparameter grid including additional optimizers
param_grid = {
    'optimizer': ['adam', 'rmsprop', 'adadelta', 'adagrad']
}

# Define a custom scorer for GridSearchCV
def mean_cosine_similarity(y_true, y_pred):
    cosine_similarities = cosine_similarity(y_true, y_pred)
    return np.mean(cosine_similarities)

cosine_similarity_scorer = make_scorer(mean_cosine_similarity, greater_is_better=True)

# Perform grid search
grid_search = GridSearchCV(estimator=autoencoder, param_grid=param_grid, scoring=cosine_similarity_scorer, cv=3)
grid_result = grid_search.fit(X_train, X_train)

# Print the best parameters and best score
print("Best Parameters: ", grid_result.best_params_)
print("Best Score: ", grid_result.best_score_)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a 

Best Parameters:  {'optimizer': 'rmsprop'}
Best Score:  0.4870285264335325


In [34]:
# Define the autoencoder architecture with increased complexity, regularization, and batch normalization
input_dim = X_train.shape[1]
encoding_dim = 64  # Adjust as needed

input_layer = Input(shape=(input_dim,))
# Add a dense layer with ReLU activation and batch normalization
encoder_layer1 = Dense(128, activation='relu')(input_layer)
encoder_layer1 = BatchNormalization()(encoder_layer1)
# Add a dropout layer for regularization
encoder_layer1 = Dropout(0.5)(encoder_layer1)

# Add another dense layer with ReLU activation and batch normalization
encoder_layer2 = Dense(encoding_dim, activation='relu')(encoder_layer1)
encoder_layer2 = BatchNormalization()(encoder_layer2)
# Add a dropout layer for regularization
encoder_layer2 = Dropout(0.5)(encoder_layer2)

decoder_layer1 = Dense(128, activation='relu')(encoder_layer2)
decoder_layer1 = BatchNormalization()(decoder_layer1)

decoder_layer2 = Dense(input_dim, activation='sigmoid')(decoder_layer1)
# Add a dropout layer for regularization
decoder_layer2 = Dropout(0.5)(decoder_layer2)

# Create the autoencoder model
autoencoder = Model(input_layer, decoder_layer2)

# Compile the autoencoder model with RMSprop optimizer
autoencoder.compile(optimizer='rmsprop', loss='mean_squared_error')

# Train the autoencoder
autoencoder.fit(X_train, X_train, epochs=50, batch_size=32, shuffle=True, validation_data=(X_test, X_test))

# Extract features using the encoder part of the autoencoder
encoder = Model(input_layer, encoder_layer2)
X_encoded_train = encoder.predict(X_train)
X_encoded_test = encoder.predict(X_test)

# Reconstruct data using the trained autoencoder
reconstructed_data = autoencoder.predict(X_test)

# Combine original test data with reconstructed data
X_test_combined = np.concatenate((X_test, reconstructed_data), axis=1)

# Compute cosine similarity between original and reconstructed data samples
cosine_similarities = cosine_similarity(X_test_combined)

# Calculate the mean cosine similarity across all samples
mean_cosine_similarity = np.mean(cosine_similarities)
print("Mean Cosine Similarity:", mean_cosine_similarity)

Epoch 1/50
69/69 [==============================] - 7s 64ms/step - loss: 0.5072 - val_loss: 0.2547
Epoch 2/50
69/69 [==============================] - 3s 47ms/step - loss: 0.5030 - val_loss: 0.2524
Epoch 3/50
69/69 [==============================] - 3s 47ms/step - loss: 0.4980 - val_loss: 0.2490
Epoch 4/50
69/69 [==============================] - 3s 48ms/step - loss: 0.4900 - val_loss: 0.2433
Epoch 5/50
69/69 [==============================] - 3s 51ms/step - loss: 0.4752 - val_loss: 0.2327
Epoch 6/50
69/69 [==============================] - 4s 52ms/step - loss: 0.4487 - val_loss: 0.2160
Epoch 7/50
69/69 [==============================] - 3s 49ms/step - loss: 0.4072 - val_loss: 0.1911
Epoch 8/50
69/69 [==============================] - 4s 51ms/step - loss: 0.3510 - val_loss: 0.1598
Epoch 9/50
69/69 [==============================] - 3s 50ms/step - loss: 0.2854 - val_loss: 0.1257
Epoch 10/50
69/69 [==============================] - 3s 49ms/step - loss: 0.2194 - val_loss: 0.0942
Epoch 11/

In [36]:
# Define the learning rate schedules
schedules = {
    'constant': lambda epoch: 0.001,
    'decreasing': lambda epoch: 0.001 * np.exp(-0.1 * epoch),
    'adaptive': lambda epoch: 0.001 if epoch < 10 else 0.0005
}

# Define a function to train the autoencoder with a given learning rate schedule
def train_autoencoder(schedule):
    # Define the autoencoder architecture
    input_dim = X_train.shape[1]
    encoding_dim = 64  # Adjust as needed

    input_layer = Input(shape=(input_dim,))
    # Add a dense layer with ReLU activation and batch normalization
    encoder_layer1 = Dense(128, activation='relu')(input_layer)
    encoder_layer1 = BatchNormalization()(encoder_layer1)
    # Add a dropout layer for regularization
    encoder_layer1 = Dropout(0.5)(encoder_layer1)

    # Add another dense layer with ReLU activation and batch normalization
    encoder_layer2 = Dense(encoding_dim, activation='relu')(encoder_layer1)
    encoder_layer2 = BatchNormalization()(encoder_layer2)
    # Add a dropout layer for regularization
    encoder_layer2 = Dropout(0.5)(encoder_layer2)

    decoder_layer1 = Dense(128, activation='relu')(encoder_layer2)
    decoder_layer1 = BatchNormalization()(decoder_layer1)

    decoder_layer2 = Dense(input_dim, activation='sigmoid')(decoder_layer1)
    # Add a dropout layer for regularization
    decoder_layer2 = Dropout(0.5)(decoder_layer2)

    # Create the autoencoder model
    autoencoder = Model(input_layer, decoder_layer2)

    # Define the optimizer with RMSprop
    optimizer = RMSprop(lr=0.001)

    # Compile the autoencoder model with RMSprop optimizer
    autoencoder.compile(optimizer=optimizer, loss='mean_squared_error')

    # Define the learning rate scheduler callback
    lr_scheduler = LearningRateScheduler(schedule)

    # Train the autoencoder with learning rate scheduler
    autoencoder.fit(X_train, X_train, epochs=50, batch_size=32, shuffle=True, validation_data=(X_test, X_test), callbacks=[lr_scheduler])

    # Extract features using the encoder part of the autoencoder
    encoder = Model(input_layer, encoder_layer2)
    X_encoded_test = encoder.predict(X_test)

    # Reconstruct data using the trained autoencoder
    reconstructed_data = autoencoder.predict(X_test)

    # Combine original test data with reconstructed data
    X_test_combined = np.concatenate((X_test, reconstructed_data), axis=1)

    # Compute cosine similarity between original and reconstructed data samples
    cosine_similarities = cosine_similarity(X_test_combined)

    # Calculate the mean cosine similarity across all samples
    mean_cosine_similarity = np.mean(cosine_similarities)
    
    return mean_cosine_similarity

# Train the autoencoder with different learning rate schedules and keep track of the best one
best_schedule = None
best_similarity = -1
for schedule_name, schedule_func in schedules.items():
    print("Training with learning rate schedule:", schedule_name)
    similarity = train_autoencoder(schedule_func)
    print("Mean Cosine Similarity:", similarity)
    if similarity > best_similarity:
        best_similarity = similarity
        best_schedule = schedule_name

print("Best learning rate schedule:", best_schedule)
print("Best Mean Cosine Similarity:", best_similarity)

Training with learning rate schedule: constant


Epoch 1/50
69/69 [==============================] - 6s 58ms/step - loss: 0.5071 - val_loss: 0.2547 - lr: 0.0010
Epoch 2/50
69/69 [==============================] - 3s 50ms/step - loss: 0.5031 - val_loss: 0.2527 - lr: 0.0010
Epoch 3/50
69/69 [==============================] - 3s 49ms/step - loss: 0.4983 - val_loss: 0.2497 - lr: 0.0010
Epoch 4/50
69/69 [==============================] - 3s 47ms/step - loss: 0.4900 - val_loss: 0.2444 - lr: 0.0010
Epoch 5/50
69/69 [==============================] - 3s 50ms/step - loss: 0.4751 - val_loss: 0.2344 - lr: 0.0010
Epoch 6/50
69/69 [==============================] - 3s 46ms/step - loss: 0.4478 - val_loss: 0.2171 - lr: 0.0010
Epoch 7/50
69/69 [==============================] - 3s 50ms/step - loss: 0.4055 - val_loss: 0.1914 - lr: 0.0010
Epoch 8/50
69/69 [==============================] - 3s 46ms/step - loss: 0.3485 - val_loss: 0.1611 - lr: 0.0010
Epoch 9/50
69/69 [==============================] - 3s 50ms/step - loss: 0.2823 - val_loss: 0.1281 - lr:

Mean Cosine Similarity: 0.39306323015021055
Training with learning rate schedule: decreasing
Epoch 1/50
69/69 [==============================] - 8s 59ms/step - loss: 0.5070 - val_loss: 0.2546 - lr: 0.0010
Epoch 2/50
69/69 [==============================] - 3s 48ms/step - loss: 0.5032 - val_loss: 0.2526 - lr: 9.0484e-04
Epoch 3/50
69/69 [==============================] - 3s 43ms/step - loss: 0.4992 - val_loss: 0.2501 - lr: 8.1873e-04
Epoch 4/50
69/69 [==============================] - 3s 44ms/step - loss: 0.4941 - val_loss: 0.2466 - lr: 7.4082e-04
Epoch 5/50
69/69 [==============================] - 3s 44ms/step - loss: 0.4867 - val_loss: 0.2417 - lr: 6.7032e-04
Epoch 6/50
69/69 [==============================] - 3s 47ms/step - loss: 0.4765 - val_loss: 0.2352 - lr: 6.0653e-04
Epoch 7/50
69/69 [==============================] - 3s 46ms/step - loss: 0.4635 - val_loss: 0.2277 - lr: 5.4881e-04
Epoch 8/50
69/69 [==============================] - 3s 45ms/step - loss: 0.4476 - val_loss: 0.2194 

Mean Cosine Similarity: 0.910163004335495
Training with learning rate schedule: adaptive
Epoch 1/50
69/69 [==============================] - 6s 53ms/step - loss: 0.5073 - val_loss: 0.2548 - lr: 0.0010
Epoch 2/50
69/69 [==============================] - 3s 45ms/step - loss: 0.5030 - val_loss: 0.2528 - lr: 0.0010
Epoch 3/50
69/69 [==============================] - 3s 48ms/step - loss: 0.4981 - val_loss: 0.2500 - lr: 0.0010
Epoch 4/50
69/69 [==============================] - 3s 44ms/step - loss: 0.4899 - val_loss: 0.2446 - lr: 0.0010
Epoch 5/50
69/69 [==============================] - 3s 47ms/step - loss: 0.4750 - val_loss: 0.2347 - lr: 0.0010
Epoch 6/50
69/69 [==============================] - 3s 45ms/step - loss: 0.4481 - val_loss: 0.2176 - lr: 0.0010
Epoch 7/50
69/69 [==============================] - 3s 47ms/step - loss: 0.4063 - val_loss: 0.1940 - lr: 0.0010
Epoch 8/50
69/69 [==============================] - 3s 46ms/step - loss: 0.3497 - val_loss: 0.1625 - lr: 0.0010
Epoch 9/50
69/6